# Save IGRINS observation times to a machine readable log

We'll need this to register the TESS observations.

### Get the IGRINS epoch times.

In [1]:
import glob

In [2]:
from astropy.io import fits

In [3]:
reduced_fns = glob.glob('../../data/IGRINS/originals/GS-2021A-DD-104/*/reduced/SDCH*spec_a0v.fits')

In [4]:
raw_fns = glob.glob('../../data/IGRINS/originals/GS-2021A-DD-104/*/SDCH*.fits')

In [5]:
len(reduced_fns)

8

In [6]:
len(raw_fns)

28

In [7]:
hdu = fits.open(raw_fns[23])[0]
hdr = hdu.header

In [8]:
keep_keys = ['OBJECT', 'EXPTIME', 'FRMTYPE', 'OBJTYPE', 'JD-OBS']

In [9]:
import pandas as pd

In [10]:
df_raw = pd.DataFrame()

for i, fn in enumerate(raw_fns):
    try:
        hdu = fits.open(fn)[0]
        hdr = hdu.header
        out_dict = {key:hdr[key] for key in keep_keys}
        out_dict['night_dir'] = fn[44:44+8]
        df = pd.DataFrame(out_dict, index=[i])
        df_raw = df_raw.append(df)
    except:
        print(f"{fn} had a problem")

In [11]:
df_log = df_raw.sort_values('JD-OBS').reset_index(drop=True)

Make a quick summary of how many spectra were acquired on each night:

In [12]:
from astropy.time import Time

In [13]:
df_log['BTJD'] = df_log['JD-OBS']- 2457000.0

In [14]:
df_out = df_log.groupby(['night_dir', 'OBJECT', 'EXPTIME']).BTJD.mean().round(decimals=4).to_frame()
df_out

BTJD
night_dir OBJECT     EXPTIME           
20210311  Luhman 16A 150.0    2285.6525
          Luhman 16B 150.0    2285.6410
20210312  Luhman 16A 150.0    2286.7599
          Luhman 16B 150.0    2286.7497
20210313  Luhman 16A 150.0    2287.6744
          Luhman 16B 150.0    2287.6566
20210317  Luhman 16A 150.0    2291.8097
          Luhman 16B 150.0    2291.8008

Neat! Let's save these so we can read them in later.

In [15]:
df_out.to_csv('../../data/IGRINS/2021A_metadata_log.csv')

In [16]:
! head ../../data/IGRINS/2021A_metadata_log.csv

night_dir,OBJECT,EXPTIME,BTJD
20210311,Luhman 16A,150.0,2285.6525
20210311,Luhman 16B,150.0,2285.641
20210312,Luhman 16A,150.0,2286.7599
20210312,Luhman 16B,150.0,2286.7497
20210313,Luhman 16A,150.0,2287.6744
20210313,Luhman 16B,150.0,2287.6566
20210317,Luhman 16A,150.0,2291.8097
20210317,Luhman 16B,150.0,2291.8008


Great!  Now we can take those dates and overlay them on TESS lightcurves.